In [0]:
df = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("/Volumes/pysparkdbt/source/source_data/customers/")

In [0]:
# Checking the schema for customers
schema_customers = df.schema
schema_customers

In [0]:
display(df)

### Spark Streaming Process

In [0]:
entities = ["customers","trips","locations","payments", "vehicles", "drivers"]

In [0]:
for entity in entities:

    df_batch = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load(f"/Volumes/pysparkdbt/source/source_data/{entity}/")

    schema_entity = df_batch.schema

    df = spark.readStream.format("csv")\
        .option("header", "true")\
        .schema(schema_entity)\
        .load(f"/Volumes/pysparkdbt/source/source_data/{entity}/")

    # Checkpoints to enable, with trigger once, we just process once to not exceeed a lot of compute, in other settings we could use processingTime = "10 seconds" or so. It dictates when we want to apply the incremental load.
    df.writeStream.format("delta")\
        .option("checkpointLocation", f"/Volumes/pysparkdbt/bronze/checkpoint/{entity}")\
        .trigger(once=True)\
        .toTable(f"pysparkdbt.bronze.{entity}")